In [2]:
import pandas as pd

countries = ['KR', 'JP', 'CA', 'GB', 'CN']
df = pd.read_csv('starbucks_2018_11_06.csv')
df = df[df.state.isin(countries)]
get_country_df = lambda df, country: df[df.state == country]

kr_df = get_country_df(df, countries[0])
jp_df = get_country_df(df, countries[1])
ca_df = get_country_df(df, countries[2])
gb_df = get_country_df(df, countries[3])
cn_df = get_country_df(df, countries[4])

def get_store_coordinates(country_df):
    import random
    name = list(country_df['name'])

    # shuffle list
    random.seed(4)
    random.shuffle(name)

    # (lat, long) switch to (long, lat)
    latitude = list(country_df['latitude'])
    longitude = list(country_df['longitude'])

    result = {}
    for i in range(len(name)):
        result[name[i]] =  (latitude[i], longitude[i])
    return result

kr_stores = get_store_coordinates(kr_df)
jp_stores = get_store_coordinates(jp_df)
ca_stores = get_store_coordinates(ca_df)
gb_stores = get_store_coordinates(gb_df)
cn_stores = get_store_coordinates(cn_df)

all_countries = [kr_stores, jp_stores, ca_stores, gb_stores, cn_stores]





def get_permutation(stores,current_best_distance, sample):
     import geopy.distance
     from  itertools import permutations
     EXPLORE = True
     clone = list(sample[:])  # create copy
     clone.append(clone[0])

     total_distance = 0
     for index in range(len(clone)-1):
         first_store = stores[clone[index]]
         second_store = stores[clone[index+1]]
         # geopy.distance.geodesic(lat, lon)
         distance = geopy.distance.geodesic(first_store, second_store).km
         total_distance += distance
         if total_distance > current_best_distance:
             EXPLORE = False
             break


     def append_central(tuple):
         result = list(tuple)
         result.append(result[0])
         return result

     if total_distance < current_best_distance and EXPLORE:
         result = list(map(append_central,permutations(sample)))
         return True, result
     else:
         return False, None



def find_optimum_route(stores={}, country="", ITER_LIMIT=1):
    from queue import PriorityQueue
    class Graph:
        def __init__(self, num_of_vertices):
            self.v = num_of_vertices
            self.edges = [[-1 for i in range(num_of_vertices)] for j in range(num_of_vertices)]
            self.visited = []
        def add_edge(self, u, v, weight):
            self.edges[u][v] = weight
            self.edges[v][u] = weight
    def dijkstra(graph, start_vertex):
        D = {v:float('inf') for v in range(graph.v)}
        D[start_vertex] = 0

        pq = PriorityQueue()
        pq.put((0, start_vertex))

        while not pq.empty():
            (dist, current_vertex) = pq.get()
            graph.visited.append(current_vertex)

            for neighbor in range(graph.v):
                if graph.edges[current_vertex][neighbor] != -1:
                    distance = graph.edges[current_vertex][neighbor]
                    if neighbor not in graph.visited:
                        old_cost = D[neighbor]
                        new_cost = D[current_vertex] + distance
                        if new_cost < old_cost:
                            pq.put((new_cost, neighbor))
                            D[neighbor] = new_cost
        return D
    iteration = []
    best = []
    import random
    import numpy as np

    import geopy.distance


    ITERATION = 0
    names = list(stores.keys())


    current_best_configuration = None
    current_best_distance = np.inf  # minimum total distance
    BAD_EXPLORE = []
    while ITERATION < ITER_LIMIT:

        current_sample = random.sample(names, 7)  # sample 7 random elements from name
        should_get_permute = True

        def check_bac_comb():
            for bad_comb in BAD_EXPLORE:
                is_a_comb = True
                for e in bad_comb:
                    if e not in current_sample:
                        is_a_comb = False
                        break
                if is_a_comb:
                    should_get_permute = False
                    break

        if should_get_permute:
            should_explore, current_config_permutations = get_permutation(stores, current_best_distance, current_sample)
            # BAD_EXPLORE.append(current_sample) # explored
        else:
            should_explore, current_config_permutations = False, None

        if should_explore and current_config_permutations is not None:
            g = Graph(8)
            for possible_config in current_config_permutations:
                total_distance = 0
                i=0
                for index in range(len(possible_config) - 1):
                    first_store = stores[possible_config[index]]
                    second_store = stores[possible_config[index + 1]]
                    # geopy.distance.geodesic(lat, lon)
                    distance = geopy.distance.geodesic(first_store, second_store).km
                    g.add_edge(index, index+1, distance)

            total_distance = dijkstra(g, 0)[7]  # invoke dijkstra
            if current_best_distance > total_distance:
                current_best_distance = total_distance
                current_best_configuration = possible_config
                iteration.append(ITERATION)
                best.append(current_best_distance)
                # print(f"iter={ITERATION}, best={current_best_distance}")
        ITERATION+=1


    return { country: current_best_configuration, "distance_km": current_best_distance, "iteration": iteration, "best_at_iteration": best}


# 86
def get_best_path_all():
    from math import comb
    c = 0
    for country in all_countries:
        # optimized_route = find_optimum_route(country, f"{countries[c]}", ITER_LIMIT=(comb(len(country), 7) / 86))
        optimized_route = find_optimum_route(country, f"{countries[c]}", ITER_LIMIT=(10000))
        print(optimized_route)
        c += 1
get_best_path_all()

{'KR': ['영풍문고점', '연세세브란스종합관', 'Gyeongbokgung Intersection', '신촌점', '구역삼사거리점', '대치점', 'Banyawol E-Mart', '영풍문고점'], 'distance_km': 22.428058667273707, 'iteration': [0, 8, 31, 115, 129, 249, 2258], 'best_at_iteration': [123.0564997279113, 115.8277039523808, 64.96263550655681, 49.415567618451924, 47.3932758464522, 27.962947997218464, 22.428058667273707]}
{'JP': ['atre Ebisu (2F)', 'TSUTAYA YOKOHAMA MINATOMIRAI', 'Shinjuku Marui Honkan 2F', 'Yamatotakada', 'Takamatsu You Me Town', 'Kusatsu Kokudo 1 go', 'Hiroshima Eki ASSE', 'atre Ebisu (2F)'], 'distance_km': 107.62854473677068, 'iteration': [0, 2, 4, 36, 152, 160, 1505, 2224], 'best_at_iteration': [1921.1121821875458, 1022.8504305630804, 534.2694916407, 418.0383798038501, 380.9770124247841, 197.3192087113714, 116.53302368661036, 107.62854473677068]}
{'CA': ['Northills SC - Kamloops', 'Safeway #8874 Sherwood Park', '235 Centennial Rd', 'Safeway #8891 Westmount, Edmonton', 'Sunterra @ Westjet Offices', 'Whyte Avenue', 'Safeway #8917 Okotoks'